In [186]:
import pandas as pd
import numpy as np
import itertools
from bs4 import BeautifulSoup
import requests
from requests import get
import time
from random import seed
from random import random
from random import randint

In [ ]:
# initialize a list called houses 
houses = []
# initialize loop variable for running through pages
count = 1
while count <= 85:
    # getting details for first page
    if count == 1:
        first_page = 'https://rentberry.com/de/apartments/s/berlin-germany'
        # request the response
        response = get(first_page)
        # parsing html 
        html_soup = BeautifulSoup(response.text, 'html.parser')
        # in the html of the page, find all the bins with <li> and class:
        house_data = html_soup.find_all('div', class_="apartment-item ng-star-inserted")
        # I like to print where the program is on the screen so we can follow its progress and where any errors happened
        print(first_page)
        
        # if the response was not empty (if something was actually scraped)
        if house_data != []:
            # add to the list houses
            houses.extend(house_data)
            # random wait times
            value = random()
            print(value)
            time.sleep(value)
    # pages other than the first
    elif count != 1:
    # collect four and wait random times 
        url = 'https://rentberry.com/de/apartments/s/berlin-germany?page=' + str(count) + '&sort=relevance'
        print(url)
        response = get(url)
        html_soup = BeautifulSoup(response.text, 'html.parser')
        print(response)
        house_data = html_soup.find_all('div', class_="apartment-item ng-star-inserted")

        if house_data != []:
            houses.extend(house_data)
            value = random()
            print(value)
            time.sleep(value)

        # if you get empty response, stop the loop
        else:
            print('empty')
            break
            

    count += 1


In [206]:
def replace_empty_string_with_NAN(x):
    if x == '':
        return np.nan
    else:
        return x

In [ ]:
# DATA FORMATTING 
## initializing lists and variables
count = 0
house_price = []
location = []
bedrooms = []
surface = []
## how long we are running the while loop for 
n = int(len(houses)) - 1
while count <= n:
    # running the loop through each html bin we scraped
    num = houses[count]
    
    # getting the price: 
    price = num.find_all('span',{"class":"label price ng-star-inserted"})[0].text
    house_price.append(price)
    df_price = pd.DataFrame({'house_price':house_price})
    df_price['house_price']=df_price['house_price'].str.replace("€","")
    df_price['house_price']=df_price['house_price'].str.replace(",","")
        
   #getting the location: 
    street = num.find('div',{"class":"apt-address font-xs one-line-string"}).text
    location.append(street)
    df_postcode = pd.DataFrame({'postcode':location})
    df_postcode['postcode'] = df_postcode['postcode'].str.replace("\D","")
    df_postcode['postcode'] = df_postcode['postcode'].str[-5:]
    df_postcode['postcode'] = df_postcode['postcode'].apply(lambda x : replace_empty_string_with_NAN(x))

    #getting the number of bedrooms: 
    bedrooms_num = num.find_all('span',{"class":"bedrooms ng-star-inserted"})[0].text
    bedrooms.append(bedrooms_num)
    df_bedrooms = pd.DataFrame({'bedrooms':bedrooms})
    
    #getting the sq meter size: 
    print(count)
    size = num.find_all('span',{"class":"space"})[0].text
    surface.append(size)
    df_surface = pd.DataFrame({'surface':surface}) 
    count += 1

# concat all the different dataframes created, culminating in dfa (completed dataframe)

result = pd.concat([df_price, df_postcode], axis=1, sort=False)
result1 = pd.concat([result, df_bedrooms], axis=1, sort=False)
dfa = pd.concat([result1, df_surface], axis=1, sort=False)
dfa=dfa.dropna()
dfa.to_csv('dfa_rentberry.csv')

In [210]:
dfa['price_per_sqm']=dfa["house_price"].astype(int)/dfa["surface"].astype(int)
dfa = dfa.reset_index()

In [211]:
X=dfa.drop('house_price',axis='columns')
y=dfa.house_price
X = X.values.astype(float)
y = y.values.astype(float)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)


0.7945205868328653

In [215]:
from sklearn.linear_model import LinearRegression
lr_clf=LinearRegression()
lr_clf.fit(X_train,y_train)
lr_clf.score(X_test,y_test)

0.7945205868328653

In [216]:
from sklearn.svm import SVC
svm=SVC()
svm.fit(X_train,y_train)
svm.score(X_test,y_test)

0.005420054200542005

In [219]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=40)
rf.fit(X_train,y_train)
rf.score(X_test,y_test)

0.11382113821138211